In [3]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as K
import numpy as np
import h5py

In [4]:
def window_stack(x,size_step,size_trayect):
    n = x.shape[0]
    list_trayect = list()
    for i in range(0,size_trayect):
        aux = x[i:1+n+i-size_trayect:size_step]
        list_trayect.append(aux)
    return np.hstack(list_trayect) 

def explained_variance_score(y_true,y_pred):
    error = y_true - y_pred
    var_error = K.sum(K.square(error - K.mean(error)))
    var_true = K.sum(K.square(y_true - K.mean(y_pred)))
    return (1 - (var_error/var_true))


def r2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return ( 1 - SS_res/(SS_tot + K.epsilon()))



def autoencoder(input_shape):
    model = Sequential(name="Autoencoder")
    # Encoder
    model.add(layers.InputLayer(input_shape= input_shape)) # 20x12x1
    model.add(layers.Conv2D(16, kernel_size=(1, 6),strides=(1,6),activation='tanh',padding='valid',name="conv_1")) # 20x2x16
    model.add(layers.Conv2D(32, kernel_size=(1, 2),activation='tanh',padding='valid',name="conv_2")) # 20x1x32
    model.add(layers.Conv2D(64, kernel_size=(3, 1),strides=(2,1),activation='tanh',padding='same',name="conv_3")) # 10x1x64
    model.add(layers.Conv2D(128, kernel_size=(2, 1),strides=(2,1),activation='tanh',padding='valid',name="conv_4")) # 5x1x128
    model.add(layers.Flatten(name="flatten_1")) # 640
    # Coding
    model.add(layers.Dense(300,activation='tanh',name="dense_1")) # 300
    model.add(layers.Dense(640,activation='tanh',name="dense_2")) # 640
    # Decoder
    model.add(layers.Reshape((5,1,128),name="reshape_1")) # 5x1x128
    model.add(layers.Conv2DTranspose(64, kernel_size=(2, 1),strides=(2,1),activation='tanh',padding='valid',name="deconv_1")) # 10x1x64
    model.add(layers.Conv2DTranspose(32,kernel_size=(3,1), strides=(2, 1), activation='tanh',padding='same',name="deconv_2")) # 20x1x32
    model.add(layers.Conv2DTranspose(16,kernel_size=(1,2), strides=(1, 2), activation='tanh',padding='valid',name="deconv_3")) # 20x2x16
    model.add(layers.Conv2DTranspose(1,kernel_size=(1,6), strides=(1, 6), activation='tanh',padding='valid',name="deconv_4")) # 20x12x1
    return model  

## Leer data de archivo hdf5

In [5]:
with h5py.File('data_gps.hdf5', 'a') as data:
    X_train = [ data["X_train"][v][:] for v in list(data["X_train"].keys())]
    X_valid = [ data["X_valid"][v][:] for v in list(data["X_valid"].keys())]


## Parametros para tranformar datos gps a series de tiempo

In [6]:
size_step, size_trayect = 39, 40 
# Cambiar forma de entrada, solo usar para redes convolucionales. Comentar esta linea en caso de usar red autoencoder apilado
height, width = 20,12 

## Transformar data train

In [7]:
X_train = [ window_stack(x,size_step, size_trayect) for x in X_train ]
X_train = np.concatenate(X_train,axis=0)
X_train = np.reshape(X_train, (len(X_train), height, width, 1))

## Transformar data valid

In [8]:
X_valid = [ window_stack(x,size_step, size_trayect) for x in X_valid ]
X_valid = np.concatenate(X_valid,axis=0)
X_valid = np.reshape(X_valid, (len(X_valid), height, width, 1))

## Train y valid

In [9]:
print("X_train : ",X_train.shape)
print("X_valid : ",X_valid.shape)

X_train :  (612846, 20, 12, 1)
X_valid :  (216738, 20, 12, 1)


## Arquitectura red

In [37]:
input_shape = X_train.shape[1:] 
model = autoencoder(input_shape)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 20, 2, 16)         112       
_________________________________________________________________
conv_2 (Conv2D)              (None, 20, 1, 32)         1056      
_________________________________________________________________
conv_3 (Conv2D)              (None, 10, 1, 64)         6208      
_________________________________________________________________
conv_4 (Conv2D)              (None, 5, 1, 128)         16512     
_________________________________________________________________
flatten_1 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               192300    
_________________________________________________________________
dens

## Entrenar red

In [ ]:
# configuracion
nb_epoch = 300
batch_size = 32
model.compile(optimizer='sgd', loss='mse',metrics=[r2,explained_variance_score])
model.fit(X_train,X_train,
          batch_size=batch_size,
          epochs=nb_epoch,
          validation_data=(X_valid,X_valid),
          callbacks=[TensorBoard(log_dir='./logs/')])
model.save("./autoencoder.h5")